In [ ]:
%matplotlib inline
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import pandas_profiling

from google.cloud import bigquery
import bq_helper

import sklearn
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from sklearn import metrics
from sklearn.linear_model import LogisticRegression

import wordcloud

In [ ]:
stackoverflow = bq_helper.BigQueryHelper("bigquery-public-data","stackoverflow")
stackoverflow.list_tables()

In [ ]:
stackoverflow.head("posts_questions")

In [ ]:
stackoverflow.table_schema("posts_questions")

In [ ]:
posts_count_query = """select EXTRACT(year FROM creation_date) AS year, sum(id) as posts
        from `bigquery-public-data.stackoverflow.posts_questions`
        where extract(year from creation_date) >= 2009 and extract(year from creation_date) < 2019
        group by year
        order by year
        LIMIT 2000000
        """

posts_count = stackoverflow.query_to_pandas(posts_count_query)
posts_count.describe()

In [ ]:
data = pd.DataFrame(posts_count)
data.head(10)

In [ ]:
pandas_profiling.ProfileReport(data)

In [ ]:
get_tags_query = """SELECT tags
         FROM `bigquery-public-data.stackoverflow.posts_questions`
         LIMIT 200000
         """
tags_df = stackoverflow.query_to_pandas_safe(get_tags_query)
tags = ' '.join(tags_df.tags).lower()

In [ ]:
cloud = wordcloud.WordCloud(background_color='white',
                            max_font_size=200,
                            width=1600,
                            height=800,
                            max_words=300,
                            relative_scaling=.5).generate(tags)
plt.figure(figsize=(20,10))
plt.axis('off')
plt.savefig('stackOverflow.png')
plt.imshow(cloud);

In [ ]:
# prepare the trainig data
pd.to_numeric(posts_count['year'])
year = posts_count['year'].values.reshape(-1,1)
posts = posts_count['posts'].values.reshape(-1,1)

# build a linear regression model
model = LinearRegression()
X_train, X_test, y_train, y_test = train_test_split(year,posts,test_size=0.2,shuffle=False)
model.fit(X_train,y_train)
predictions = model.predict(X_test)
print('Predicted values: ',predictions)

# compute the mean squared error
print('Mean Squared Error:', metrics.mean_squared_error(y_test, predictions))

In [ ]:
# plot the training, testing and predicted points
plt.scatter(X_train,y_train, color = "blue")
plt.scatter(X_test, y_test, color = "green")
plt.plot(X_test, predictions, color = "red")
plt.gca().legend(('Y-Predicted','Y-Train', 'Y-Test'))
plt.title('Total posts count vs years')
plt.xlabel('Year')
plt.ylabel('Posts')
plt.show()

Compute the future predictions of the tags related to front end development 

In [ ]:
# fetch the records with tags, angularjs,bootstrap,php,html,javascript,css
query = """select EXTRACT(year FROM creation_date) AS year, sum(id) as posts
        from `bigquery-public-data.stackoverflow.posts_questions`
        where extract(year from creation_date) >=2009 and extract(year from creation_date) < 2019 and (tags like '%bootstrap%' or 
        tags like '%angularjs%' or tags like '%php%' or tags like '%html%' or tags like '%javascript%' or tags like '%css%')
        group by year
        order by year
        """

frontend_posts = stackoverflow.query_to_pandas(query)
frontend_posts['posts']= frontend_posts['posts']*100/posts_count['posts']
frontend_posts

In [ ]:
frontend_posts.describe()

In [ ]:
# prepare the training data
pd.to_numeric(frontend_posts['year'])
year = frontend_posts['year'].values.reshape(-1,1)
posts = frontend_posts['posts'].values.reshape(-1,1)

# train a Linear regression model
X_train, X_test, y_train, y_test = train_test_split(year,posts,test_size=0.2,shuffle=False)
model=LinearRegression()
model.fit(X_train,y_train)
frontend_predictions = model.predict(X_test)
print('Predicted Values: ',frontend_predictions)

In [ ]:
# plot a graph
plt.scatter(X_train,y_train, color = "blue")
plt.scatter(X_test, y_test, color = "green")
plt.plot(X_test, frontend_predictions, color = "red")
plt.gca().legend(('Y-Predicted','Y-Train', 'Y-Test'))
plt.title('Front-end development posts count vs years')
plt.xlabel('Year')
plt.ylabel('Posts')
plt.show()

Compute the future predictions of the tags related to databases

In [ ]:
# fetch the records with tags, mysql,mongodb,nosql,postgresql,cassandra
query = """select EXTRACT(year FROM creation_date) AS year, sum(id) as posts
        from `bigquery-public-data.stackoverflow.posts_questions`
        where extract(year from creation_date)>= 2009 and extract(year from creation_date) < 2019 
        and (tags like '%mysql%' or tags like '%nosql%' or tags like '%mongodb%' 
        or tags like '%postgresql%' or tags like '%cassandra%')
        group by year
        order by year
        """

database_posts = stackoverflow.query_to_pandas(query)
database_posts['posts']= database_posts['posts']*100/posts_count['posts']
database_posts

In [ ]:
database_posts.describe()

In [ ]:
# prepare the training data
pd.to_numeric(database_posts['year'])
year = database_posts['year'].values.reshape(-1,1)
posts = database_posts['posts'].values.reshape(-1,1)

In [ ]:
# train a Linear regression model
X_train, X_test, y_train, y_test = train_test_split(year,posts,test_size=0.2,shuffle=False)
model=LinearRegression()
model.fit(X_train,y_train)
database_predictions = model.predict(X_test)
print('Predicted Values: ',database_predictions)

In [ ]:
# plot a graph
plt.scatter(X_train,y_train, color = "blue")
plt.scatter(X_test, y_test, color = "green")
plt.plot(X_test, database_predictions, color = "red")
plt.gca().legend(('Y-Predicted','Y-Train', 'Y-Test'))
plt.title('Database development posts count vs years')
plt.xlabel('Year')
plt.ylabel('Posts')
plt.show()